In [3]:
!pip install numpy==1.26.4 librosa==0.10.2.post1 tqdm==4.67.1 pandas==2.2.2 joblib==1.4.2 scikit-learn==1.5.0 tensorflow==2.17.1 python_speech_features==0.6 scikeras==0.10.0 matplotlib seaborn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 79.6 MB/s eta 0:00:00:00:01:01


In [4]:
# ------------------ Import Libraries ------------------
import os
import numpy as np
import librosa
from tqdm import tqdm
import pandas as pd
import warnings
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    roc_curve,
    auc
)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    VotingClassifier
)
from sklearn.naive_bayes import GaussianNB

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping

from python_speech_features import logfbank, fbank

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

In [5]:
# ------------------ Feature Extraction Functions ------------------
def pad_or_truncate(features, target_length):
    """
    Pads or truncates the feature matrix to ensure uniform length.

    Parameters:
    - features (np.ndarray): Feature matrix of shape (Time, Features).
    - target_length (int): Desired number of time frames.

    Returns:
    - np.ndarray: Padded or truncated feature matrix.
    """
    if features.shape[0] < target_length:
        padding = target_length - features.shape[0]
        padded_features = np.pad(features, ((0, padding), (0, 0)), mode='constant')
        return padded_features
    else:
        return features[:target_length, :]

def extract_filterbank_energies(audio, samplerate=44100, nfilt=40):
    """
    Extracts Filterbank Energies from the audio signal.

    Parameters:
    - audio (np.ndarray): Audio time series.
    - samplerate (int): Sampling rate.
    - nfilt (int): Number of filterbanks.

    Returns:
    - np.ndarray: Filterbank energy features.
    """
    features, _ = fbank(audio, samplerate, nfilt=nfilt)
    return features

def extract_log_filterbank_energies(audio, samplerate=44100, nfilt=40):
    """
    Extracts Log Filterbank Energies from the audio signal.

    Parameters:
    - audio (np.ndarray): Audio time series.
    - samplerate (int): Sampling rate.
    - nfilt (int): Number of filterbanks.

    Returns:
    - np.ndarray: Log Filterbank energy features.
    """
    features = logfbank(audio, samplerate, nfilt=nfilt)  # Remove unpacking
    return features

def extract_spectral_subband_centroids(audio, samplerate=44100, nfilt=40):
    """
    Extracts Spectral Subband Centroids from the audio signal.

    Parameters:
    - audio (np.ndarray): Audio time series.
    - samplerate (int): Sampling rate.
    - nfilt (int): Number of filterbanks.

    Returns:
    - np.ndarray: Spectral Subband Centroids features.
    """
    filter_banks, _ = fbank(audio, samplerate, nfilt=nfilt)
    centroids = np.zeros(filter_banks.shape)
    for i in range(filter_banks.shape[0]):
        if np.sum(filter_banks[i]) != 0:
            centroids[i] = np.sum(filter_banks[i] * np.arange(1, nfilt + 1)) / np.sum(filter_banks[i])
        else:
            centroids[i] = 0
    return centroids

def extract_spncc(audio, samplerate=44100, nfilt=40, ncep=13):
    """
    Extracts Power-Normalized Cepstral Coefficients (SPNCC) from the audio signal.

    Parameters:
    - audio (np.ndarray): Audio time series.
    - samplerate (int): Sampling rate.
    - nfilt (int): Number of filterbanks.
    - ncep (int): Number of cepstral coefficients.

    Returns:
    - np.ndarray: SPNCC features.
    """
    filter_banks = extract_filterbank_energies(audio, samplerate, nfilt)
    power = np.sum(filter_banks, axis=1)
    power_normalized = filter_banks / (power[:, np.newaxis] + 1e-10)
    spncc = librosa.feature.mfcc(S=np.log(power_normalized + 1e-10), n_mfcc=ncep).T
    return spncc

def extract_msrcc(audio, samplerate=44100, nfilt=40, ncep=13):
    """
    Extracts Magnitude-based Spectral Root Cepstral Coefficients (MSRCC) from the audio signal.

    Parameters:
    - audio (np.ndarray): Audio time series.
    - samplerate (int): Sampling rate.
    - nfilt (int): Number of filterbanks.
    - ncep (int): Number of cepstral coefficients.

    Returns:
    - np.ndarray: MSRCC features.
    """
    filter_banks = extract_filterbank_energies(audio, samplerate, nfilt)
    root_power_spectrum = np.sqrt(filter_banks + 1e-10)
    msrcc = librosa.feature.mfcc(S=np.log(root_power_spectrum + 1e-10), n_mfcc=ncep).T
    return msrcc

In [6]:
# ------------------ Dataset Loader Class ------------------
class DysarthriaDataset:
    def __init__(
        self,
        data_path,
        severity_mapping,
        sr=44100,
        target_length=128,
        feature_type='logfbank',
        n_mfcc=13,
        nfilt=40
    ):
        """
        Initializes the dataset loader.

        Parameters:
        - data_path (str): Path to the dataset directory.
        - severity_mapping (dict): Mapping of severity levels to integer labels.
        - sr (int): Sampling rate for audio files.
        - target_length (int): Target length for feature matrices.
        - feature_type (str): Type of feature to extract ('fbank', 'logfbank', 'spncc', 'msrcc', 'subband_centroid').
        - n_mfcc (int): Number of MFCC coefficients to extract (for SPNCC and MSRCC).
        - nfilt (int): Number of filterbanks.
        """
        self.data_path = data_path
        self.severity_mapping = severity_mapping
        self.sr = sr
        self.target_length = target_length
        self.feature_type = feature_type
        self.n_mfcc = n_mfcc
        self.nfilt = nfilt
        self.data = []
        self.labels = []

        for severity, label in severity_mapping.items():
            folder_path = os.path.join(data_path, severity)
            if not os.path.exists(folder_path):
                print(f"Warning: Folder path {folder_path} does not exist. Skipping.")
                continue
            for file in os.listdir(folder_path):
                if file.lower().endswith((".wav", ".mp3", ".flac")):
                    self.data.append(os.path.join(folder_path, file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves the feature vector and label for a given index.

        Parameters:
        - idx (int): Index of the sample.

        Returns:
        - tuple: (feature_vector (np.ndarray), label (int))
        """
        file_path = self.data[idx]
        label = self.labels[idx]
        audio, _ = librosa.load(file_path, sr=self.sr)

        # Extract features based on feature_type
        if self.feature_type == 'fbank':
            features = extract_filterbank_energies(audio, self.sr, self.nfilt)
        elif self.feature_type == 'logfbank':
            features = extract_log_filterbank_energies(audio, self.sr, self.nfilt)
        elif self.feature_type == 'subband_centroid':
            features = extract_spectral_subband_centroids(audio, self.sr, self.nfilt)
        elif self.feature_type == 'spncc':
            features = extract_spncc(audio, self.sr, self.nfilt, self.n_mfcc)
        elif self.feature_type == 'msrcc':
            features = extract_msrcc(audio, self.sr, self.nfilt, self.n_mfcc)
        else:
            raise ValueError(f"Unsupported feature type: {self.feature_type}")

        features_fixed = pad_or_truncate(features, self.target_length)

        # Aggregate features
        if self.feature_type in ['fbank', 'logfbank', 'subband_centroid', 'spncc', 'msrcc']:
            # Calculate mean and std across time frames
            feature_mean = features_fixed.mean(axis=0)
            feature_std = features_fixed.std(axis=0)
            # Concatenate mean and std
            feature_vector = np.concatenate([feature_mean, feature_std])
        else:
            feature_vector = features_fixed.flatten()

        return feature_vector, label

    def get_all_features_labels(self):
        """
        Extracts features and labels for the entire dataset.

        Returns:
        - tuple: (X (np.ndarray), y (np.ndarray))
        """
        X = []
        y = []
        for idx in tqdm(range(len(self)), desc=f"Extracting {self.feature_type} Features"):
            features, label = self[idx]
            X.append(features)
            y.append(label)
        return np.array(X), np.array(y)

In [7]:
# ------------------ ANN Model Creation Function ------------------
def create_ann_model(
    input_dim,
    hidden_layers=[64, 32],
    dropout_rate=0.5,
    activation='relu',
    optimizer='adam'
):
    """
    Creates a TensorFlow Keras ANN model with early stopping.

    Parameters:
    - input_dim (int): Number of input features.
    - hidden_layers (list): List containing the number of neurons in each hidden layer.
    - dropout_rate (float): Dropout rate for regularization.
    - activation (str): Activation function for hidden layers.
    - optimizer (str): Optimizer for training.

    Returns:
    - tf.keras.Model: Compiled Keras model.
    """
    model = Sequential()
    model.add(Dense(hidden_layers[0], input_dim=input_dim, activation=activation))
    model.add(Dropout(dropout_rate))

    for neurons in hidden_layers[1:]:
        model.add(Dense(neurons, activation=activation))
        model.add(Dropout(dropout_rate))

    model.add(Dense(4, activation='softmax', dtype='float32'))  # Ensure output layer is float32

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    return model

# ------------------ Visualization Functions ------------------
def plot_training_history(history, clf_name, feature_type, n_mfcc, save_dir='plots/ann'):
    """
    Plots training and validation loss and accuracy over epochs.

    Parameters:
    - history (History): Keras History object.
    - clf_name (str): Name of the classifier.
    - feature_type (str): Type of feature used.
    - n_mfcc (int): Number of MFCCs.
    - save_dir (str): Directory to save the plots.
    """
    os.makedirs(save_dir, exist_ok=True)

    # Plot Loss
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Loss Curve for {clf_name} ({feature_type}, MFCC={n_mfcc})')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    loss_plot_path = os.path.join(save_dir, f"{clf_name}_{feature_type}_mfcc{n_mfcc}_loss.png")
    plt.savefig(loss_plot_path)
    plt.close()

    # Plot Accuracy
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Accuracy Curve for {clf_name} ({feature_type}, MFCC={n_mfcc})')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    acc_plot_path = os.path.join(save_dir, f"{clf_name}_{feature_type}_mfcc{n_mfcc}_accuracy.png")
    plt.savefig(acc_plot_path)
    plt.close()

def plot_classification_report(y_true, y_pred, clf_name, feature_type, n_mfcc, save_dir='plots/classification_reports'):
    """
    Plots the classification report as a heatmap.

    Parameters:
    - y_true (np.ndarray): True labels.
    - y_pred (np.ndarray): Predicted labels.
    - clf_name (str): Name of the classifier.
    - feature_type (str): Type of feature used.
    - n_mfcc (int): Number of MFCCs.
    - save_dir (str): Directory to save the plots.
    """
    os.makedirs(save_dir, exist_ok=True)
    report = classification_report(y_true, y_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    plt.figure(figsize=(10, 8))
    sns.heatmap(report_df.iloc[:-3, :].T, annot=True, cmap='Blues')
    plt.title(f'Classification Report for {clf_name} ({feature_type}, MFCC={n_mfcc})')
    plt.xlabel('Metrics')
    plt.ylabel('Classes')
    report_plot_path = os.path.join(save_dir, f"{clf_name}_{feature_type}_mfcc{n_mfcc}_classification_report.png")
    plt.savefig(report_plot_path)
    plt.close()

def plot_confusion_matrix_matrix(y_true, y_pred, clf_name, feature_type, n_mfcc, save_dir='plots/confusion_matrices'):
    """
    Plots the confusion matrix as a heatmap.

    Parameters:
    - y_true (np.ndarray): True labels.
    - y_pred (np.ndarray): Predicted labels.
    - clf_name (str): Name of the classifier.
    - feature_type (str): Type of feature used.
    - n_mfcc (int): Number of MFCCs.
    - save_dir (str): Directory to save the plots.
    """
    os.makedirs(save_dir, exist_ok=True)
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
    plt.title(f'Confusion Matrix for {clf_name} ({feature_type}, MFCC={n_mfcc})')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    cm_plot_path = os.path.join(save_dir, f"{clf_name}_{feature_type}_mfcc{n_mfcc}_confusion_matrix.png")
    plt.savefig(cm_plot_path)
    plt.close()

def plot_results_overview(results_df, metric='Accuracy', save_dir='plots/overview'):
    """
    Plots an overview of a specific metric across different classifiers and MFCCs.

    Parameters:
    - results_df (pd.DataFrame): DataFrame containing the results.
    - metric (str): The metric to plot ('Accuracy', 'Precision', 'Recall', 'F1-Score').
    - save_dir (str): Directory to save the plots.
    """
    os.makedirs(save_dir, exist_ok=True)
    plt.figure(figsize=(12, 8))
    sns.barplot(data=results_df, x='Classifier', y=metric, hue='MFCCs')
    plt.title(f'{metric} Comparison Across Classifiers and MFCCs')
    plt.xlabel('Classifier')
    plt.ylabel(metric)
    plt.xticks(rotation=45)
    plt.legend(title='MFCCs')
    plt.tight_layout()
    overview_plot_path = os.path.join(save_dir, f"overview_{metric.lower()}.png")
    plt.savefig(overview_plot_path)
    plt.close()

def plot_roc_curve_multiclass(y_true, y_score, clf_name, feature_type, n_mfcc, save_dir='plots/roc_curves'):
    """
    Plots the ROC curve for multi-class classifiers using a one-vs-rest approach.

    Parameters:
    - y_true (np.ndarray): True labels.
    - y_score (np.ndarray): Predicted scores or probabilities.
    - clf_name (str): Name of the classifier.
    - feature_type (str): Type of feature used.
    - n_mfcc (int): Number of MFCCs.
    - save_dir (str): Directory to save the plots.
    """
    os.makedirs(save_dir, exist_ok=True)
    classes = np.unique(y_true)
    y_true_binarized = label_binarize(y_true, classes=classes)
    n_classes = y_true_binarized.shape[1]

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_binarized.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    plt.figure(figsize=(10, 8))
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], lw=2, label=f'Class {i} (AUC = {roc_auc[i]:0.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curves for {clf_name} ({feature_type}, MFCC={n_mfcc})')
    plt.legend(loc='lower right')
    roc_plot_path = os.path.join(save_dir, f"{clf_name}_{feature_type}_mfcc{n_mfcc}_roc_curve.png")
    plt.savefig(roc_plot_path)
    plt.close()

In [8]:
# ------------------ Main Pipeline Function ------------------
def main_pipeline():
    # ------------------ Configuration ------------------
    # Define severity mapping (Folder Name to Label)
    severity_mapping = {
        "VERY LOW": 0,
        "LOW": 1,
        "MEDIUM": 2,
        "HIGH": 3
    }

    # Update this path to point to your dataset directory
    dataset_path = "/kaggle/input/class-data/class-data"  # Example: "/kaggle/input/dysarthria-data/noisereduced-uaspeech"

    # Define the list of MFCCs to evaluate
    mfcc_list = [13, 26, 39, 52]

    # Define target length for feature matrices
    target_length = 128

    # Define random state for reproducibility
    RANDOM_STATE = 42

    # Define the path for the CSV file
    results_csv_path = "classifier_results.csv"

    # Initialize or load existing results
    if os.path.exists(results_csv_path):
        print(f"Loading existing results from {results_csv_path}...")
        results_df = pd.read_csv(results_csv_path)
    else:
        print("Initializing a new results DataFrame.")
        results_df = pd.DataFrame(columns=["MFCCs", "Feature_Type", "Classifier", "Accuracy", "Precision", "Recall", "F1-Score"])

    # Iterate over different numbers of MFCCs
    for n_mfcc in mfcc_list:
        print(f"\n=== Processing with {n_mfcc} MFCCs ===")

        # Define list of feature types to evaluate separately
        feature_types = ['msrcc', 'fbank', 'logfbank', 'subband_centroid', 'spncc']

        for feature_type in feature_types:
            print(f"\n--- Extracting and Evaluating Feature: {feature_type} ---")

            # Initialize dataset with current feature type and MFCC
            dataset = DysarthriaDataset(
                data_path=dataset_path,
                severity_mapping=severity_mapping,
                sr=44100,
                target_length=target_length,
                feature_type=feature_type,
                n_mfcc=n_mfcc,
                nfilt=40
            )

            # Extract features and labels
            X, y = dataset.get_all_features_labels()

            print(f"Feature matrix shape: {X.shape}")
            print(f"Labels distribution: {np.bincount(y)}")

            # ------------------ Dataset Splitting ------------------
            # Split data into Train (70%), Validation (15%), Test (15%)
            X_train, X_temp, y_train, y_temp = train_test_split(
                X, y, test_size=0.30, stratify=y, random_state=RANDOM_STATE
            )
            X_val, X_test, y_val, y_test = train_test_split(
                X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=RANDOM_STATE
            )

            print(f"Train set: {X_train.shape[0]} samples")
            print(f"Validation set: {X_val.shape[0]} samples")
            print(f"Test set: {X_test.shape[0]} samples")

            # ------------------ Classifier Definitions and Hyperparameter Grids ------------------
            # Define classifiers and their hyperparameter grids
            classifiers = {
                "Logistic Regression": {
                    "model": LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
                    "params": {
                        "classifier__C": [0.1, 1, 10, 100],
                        "classifier__penalty": ['l2'],
                        "classifier__solver": ['lbfgs']
                    }
                },
                "Support Vector Machine": {
                    "model": SVC(probability=True, random_state=RANDOM_STATE),
                    "params": {
                        "classifier__C": [0.1, 1, 10],
                        "classifier__gamma": ['scale', 'auto'],
                        "classifier__kernel": ['rbf', 'poly']
                    }
                },
                "k-Nearest Neighbors": {
                    "model": KNeighborsClassifier(),
                    "params": {
                        "classifier__n_neighbors": [3, 5, 7],
                        "classifier__weights": ['uniform', 'distance'],
                        "classifier__metric": ['euclidean', 'manhattan']
                    }
                },
                "Decision Tree": {
                    "model": DecisionTreeClassifier(random_state=RANDOM_STATE),
                    "params": {
                        "classifier__max_depth": [None, 10, 20, 30],
                        "classifier__min_samples_split": [2, 5, 10],
                        "classifier__criterion": ['gini', 'entropy']
                    }
                },
                "Random Forest": {
                    "model": RandomForestClassifier(random_state=RANDOM_STATE),
                    "params": {
                        "classifier__n_estimators": [100, 200],
                        "classifier__max_depth": [None, 10, 20],
                        "classifier__min_samples_split": [2, 5],
                        "classifier__criterion": ['gini', 'entropy']
                    }
                },
                "Gradient Boosting": {
                    "model": GradientBoostingClassifier(random_state=RANDOM_STATE),
                    "params": {
                        "classifier__n_estimators": [100, 200],
                        "classifier__learning_rate": [0.01, 0.1],
                        "classifier__max_depth": [3, 5],
                        "classifier__subsample": [0.8, 1.0]
                    }
                },
                "Naive Bayes": {
                    "model": GaussianNB(),
                    "params": {
                        "classifier__var_smoothing": [1e-9, 1e-8, 1e-7]
                    }
                },
                "Artificial Neural Network (ANN)": {
                    "model": KerasClassifier(
                        model=create_ann_model,
                        input_dim=X_train.shape[1],
                        hidden_layers=[64, 32],
                        dropout_rate=0.5,
                        activation='relu',
                        optimizer='adam',
                        epochs=100,
                        batch_size=32,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
                        verbose=0
                    ),
                    "params": {
                        "classifier__hidden_layers": [[64, 32], [128, 64, 32]],
                        "classifier__dropout_rate": [0.3, 0.5],
                        "classifier__activation": ['relu', 'tanh'],
                        "classifier__optimizer": ['adam', 'rmsprop'],
                        "classifier__batch_size": [32, 64],
                        "classifier__epochs": [50, 100],
                        "classifier__callbacks": [[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]]
                    }
                }
            }

            # ------------------ Training, Evaluation, and Result Storage ------------------
            for clf_name, clf_info in classifiers.items():
                # Skip already processed classifiers for current feature and MFCC setting
                if (
                    ((results_df['MFCCs'] == n_mfcc) & 
                    (results_df['Feature_Type'] == feature_type) &
                    (results_df['Classifier'] == clf_name)).any()
                ):
                    print(f"Skipping {clf_name} with {feature_type} and {n_mfcc} MFCCs as it has already been processed.")
                    continue

                print(f"\n--- Training {clf_name} with {feature_type} and {n_mfcc} MFCCs ---")

                # Create pipeline
                pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    ('classifier', clf_info['model'])
                ])

                # Define GridSearchCV
                grid = GridSearchCV(
                    estimator=pipeline,
                    param_grid=clf_info['params'],
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
                    scoring='accuracy',
                    n_jobs=-1,
                    verbose=0
                )

                # Fit GridSearchCV
                grid.fit(X_train, y_train)

                # Best estimator
                best_estimator = grid.best_estimator_
                best_params = grid.best_params_
                best_score = grid.best_score_

                print(f"Best Parameters for {clf_name}: {best_params}")
                print(f"Best Cross-Validation Accuracy: {best_score * 100:.2f}%")

                # Evaluate on Test Set
                y_pred = best_estimator.predict(X_test)
                test_accuracy = accuracy_score(y_test, y_pred) * 100
                test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(
                    y_test, y_pred, average='weighted'
                )

                print(f"Test Accuracy: {test_accuracy:.2f}%")
                print(f"Test Precision: {test_precision * 100:.2f}%")
                print(f"Test Recall: {test_recall * 100:.2f}%")
                print(f"Test F1-Score: {test_f1 * 100:.2f}%")

                # Append results to DataFrame
                new_row = pd.DataFrame([{
                    "MFCCs": n_mfcc,
                    "Feature_Type": feature_type,
                    "Classifier": clf_name,
                    "Accuracy": f"{test_accuracy:.2f}%",
                    "Precision": f"{test_precision * 100:.2f}%",
                    "Recall": f"{test_recall * 100:.2f}%",
                    "F1-Score": f"{test_f1 * 100:.2f}%"
                }])
                results_df = pd.concat([results_df, new_row], ignore_index=True)

                # Save results to CSV
                results_df.to_csv(results_csv_path, index=False)
                print(f"Results for {clf_name} with {feature_type} and {n_mfcc} MFCCs saved to {results_csv_path}.")

                # ------------------ Visualization ------------------
                # Plot classification report
                plot_classification_report(y_test, y_pred, clf_name, feature_type, n_mfcc)

                # Plot confusion matrix
                plot_confusion_matrix_matrix(y_test, y_pred, clf_name, feature_type, n_mfcc)

                # If the classifier is ANN, plot training history
                if clf_name == "Artificial Neural Network (ANN)":
                    # Extract the history from scikeras
                    if hasattr(best_estimator.named_steps['classifier'], 'model_') and hasattr(best_estimator.named_steps['classifier'].model_, 'history'):
                        history = best_estimator.named_steps['classifier'].model_.history
                        plot_training_history(history, clf_name, feature_type, n_mfcc)
                    else:
                        print("No training history available for ANN.")

                # Plot ROC Curve if the classifier provides probability estimates
                if hasattr(best_estimator.named_steps['classifier'], "predict_proba"):
                    y_score = best_estimator.predict_proba(X_test)
                    plot_roc_curve_multiclass(y_test, y_score, clf_name, feature_type, n_mfcc)
                elif hasattr(best_estimator.named_steps['classifier'], "decision_function"):
                    # For classifiers like SVM without predict_proba
                    y_score = best_estimator.decision_function(X_test)
                    # If y_score is 1D, reshape it
                    if len(y_score.shape) == 1:
                        y_score = np.vstack([-y_score, y_score]).T
                    plot_roc_curve_multiclass(y_test, y_score, clf_name, feature_type, n_mfcc)
                else:
                    print(f"Classifier {clf_name} does not support probability estimates for ROC curve.")

                # ------------------ Save the Best Model ------------------
                model_save_path = f"models/{clf_name.replace(' ', '')}_{feature_type}_mfcc{n_mfcc}.joblib"
                os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
                joblib.dump(best_estimator, model_save_path)
                print(f"Model saved to {model_save_path}.")

    # ------------------ Aggregated Results Visualization Function ------------------
    def aggregate_and_plot_results(results_csv_path):
        """
        Aggregates results from the CSV and plots overview metrics.

        Parameters:
        - results_csv_path (str): Path to the results CSV file.
        """
        if not os.path.exists(results_csv_path):
            print(f"Results CSV file {results_csv_path} does not exist.")
            return

        results_df = pd.read_csv(results_csv_path)

        # Convert percentage strings to float
        for metric in ["Accuracy", "Precision", "Recall", "F1-Score"]:
            results_df[metric] = results_df[metric].str.rstrip('%').astype(float)

        # Plot Accuracy Overview
        plot_results_overview(results_df, metric='Accuracy')

        # Plot Precision Overview
        plot_results_overview(results_df, metric='Precision')

        # Plot Recall Overview
        plot_results_overview(results_df, metric='Recall')

        # Plot F1-Score Overview
        plot_results_overview(results_df, metric='F1-Score')

        print("\nAggregated result visualizations have been saved.")

    # ------------------ Execute Main Pipeline ------------------
    main_pipeline()

    # ------------------ Execute Aggregated Plots ------------------
    aggregate_and_plot_results("classifier_results.csv")

In [ ]:
# ------------------ Execute Script ------------------
if __name__ == "__main__":
    main_pipeline()

Initializing a new results DataFrame.

=== Processing with 13 MFCCs ===

--- Extracting and Evaluating Feature: msrcc ---


Extracting msrcc Features: 100%|██████████| 11437/11437 [03:32<00:00, 53.74it/s]


Feature matrix shape: (11437, 26)
Labels distribution: [3825 2281 2295 3036]
Train set: 8005 samples
Validation set: 1716 samples
Test set: 1716 samples

--- Training Logistic Regression with msrcc and 13 MFCCs ---
Best Parameters for Logistic Regression: {'classifier__C': 100, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}
Best Cross-Validation Accuracy: 54.85%
Test Accuracy: 55.83%
Test Precision: 54.51%
Test Recall: 55.83%
Test F1-Score: 53.59%
Results for Logistic Regression with msrcc and 13 MFCCs saved to classifier_results.csv.
Model saved to models/LogisticRegression_msrcc_mfcc13.joblib.

--- Training Support Vector Machine with msrcc and 13 MFCCs ---
